In [2]:

#HJI
import urllib3
import json


In [6]:
f = open("조커리뷰.txt",'r',encoding='UTF-8')
형태소카운터 = {}
name = f.name

In [7]:
switch = True
while(switch):
    댓글뭉치 = ''
    for i in range(50):
        text = f.readline()
        댓글뭉치 += text
        
    if text == '':
        switch = False
    elif text == '\n':
        continue
    else:
        openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
        accessKey = "3314db0e-ee18-4aff-bd16-e85e2a503569"
        analysisCode = "srl"
        text = 댓글뭉치

        requestJson = {
            "access_key": accessKey,
            "argument": {
                "text": text,
                "analysis_code": analysisCode
            }
        }

        http = urllib3.PoolManager()
        response = http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8"},
            body=json.dumps(requestJson)
        )

        a = response.data.decode('utf-8')
        result = json.loads(a)
        
        for l in result['return_object']['sentence']:
            for i in l['morp']: 
                if i['lemma'] not in 형태소카운터.keys() and (i['type']=='NNP' or i['type']=='NNG') :
                    형태소카운터[i['lemma']] = [1,i['type']]
                elif i['lemma'] in 형태소카운터.keys() and (i['type']=='NNP' or i['type']=='NNG'):
                    형태소카운터[i['lemma']][0]+=1



In [ ]:
#top 15단어 추출 및 워드크라우드 위한 텍스트 정렬
import random
나열하기 = []
갯수들 = []

key = list(형태소카운터.keys())
for i in key:
    for j in range(형태소카운터[i][0]):
        나열하기.append(i)
    갯수들.append(형태소카운터[i][0])

갯수들.sort(reverse=True)
탑15 = 갯수들[:15]
Keywords = []
for i in 탑15:
    for k in 형태소카운터:
        if 형태소카운터[k][0] == i:
            Keywords.append(k)
        else:
            pass
random.shuffle(나열하기)
나열한텍스트 = ''
for i in 나열하기:
    나열한텍스트 += (i+' ')

In [ ]:

switch = True
f = open("조커리뷰.txt",'r',encoding='UTF-8')
topkeyworddep = {}
while(switch):
    댓글뭉치 = ''
    for i in range(50):
        text = f.readline()
        댓글뭉치 += text
    if text == '':
        switch = False
    elif text == '\n':
        continue
    else:
        openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
        accessKey = "3314db0e-ee18-4aff-bd16-e85e2a503569"
        analysisCode = "srl"
        text = 댓글뭉치

        requestJson = {
            "access_key": accessKey,
            "argument": {
                "text": text,
                "analysis_code": analysisCode
            }
        }

        http = urllib3.PoolManager()
        response = http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8"},
            body=json.dumps(requestJson)
        )

        a = response.data.decode('utf-8')
        b = json.loads(a)
        
        
        
        
        # sentence = 50문장 중 순차적 문장 하나
        for sentence in b['return_object']['sentence']:
            dependency = sentence['dependency']
            idtextdic = {}
            topidlist = []
            for i in dependency:
                if i['text'] in Keywords:
                    topidlist.append(i['id'])
                else:
                    pass
                idtextdic[i['id']] = i['text']
            for i in topidlist:
                i = int(i)
                j = dependency[i]
                if j['text'] not in topkeyworddep.keys():
                    topkeyworddep[j['text']] = []
                    for k in j['mod']:
                        topkeyworddep[j['text']].append(idtextdic[k])
                    if j['head'] == -1.0:
                        pass
                    else:
                        topkeyworddep[j['text']].append(idtextdic[j['head']])

                else:
                    for k in j['mod']:
                        topkeyworddep[j['text']].append(idtextdic[k])
                    if j['head'] == -1.0:
                        pass
                    else:
                        topkeyworddep[j['text']].append(idtextdic[j['head']])

In [ ]:
저장 = str(topkeyworddep)
with open('{}분석결과.txt'.format(name),'w',encoding = "UTF-8") as f:
    f.write(나열한텍스트+"\n"+저장)